In [11]:
# imports:
import numpy as np
from sklearn.utils import check_array, extmath
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.model_selection import train_test_split
import scipy.io as sio
from sklearn.utils import shuffle
import scipy as sp
from ELLA import ELLA
from sklearn.linear_model import Ridge, LinearRegression, LogisticRegression
from scipy.linalg import norm
import numpy as np
from scipy.io import loadmat
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

In [12]:
PATH_data = 'example_data/'

In [13]:
fullX=np.loadtxt(PATH_data + "1repeat_%dtask_Xtrain.txt"%(1))
fullY=np.loadtxt(PATH_data + "1repeat_%dtask_Ytrain.txt"%(1))

In [4]:
np.array(fullX).shape, np.array(fullY).shape 

((500, 25), (500,))

In [14]:
def get_data_Leavire(task): 
    fullX=np.loadtxt(PATH_data + "1repeat_%dtask_Xtrain.txt"%(task))
    fullY=np.loadtxt(PATH_data + "1repeat_%dtask_Ytrain.txt"%(task))
    testX = np.loadtxt(PATH_data + "1repeat_%dtask_Xtest.txt"%(task))
    testY = np.loadtxt(PATH_data + "1repeat_%dtask_Ytest.txt"%(task))
    return [fullX, fullY, testX, testY]

In [44]:
#dim = 25
ntasks = 8 # nb de taches/produits 
data = []
labels = []
test_data = []
test_labels = []
    
for i in np.arange(1, ntasks+1):
    [fullX, fullY, testX, testY] = get_data_Leavire(i)
    data.append(fullX)
    labels.append(fullY)
    test_data.append(testX)
    test_labels.append(testY)

In [7]:
np.array(data).shape, np.array(labels).shape,  np.array(test_data).shape, np.array(test_labels).shape 

((8, 500, 25), (8, 500), (8, 100, 25), (8, 100))

#### Online Multi tache

In [45]:
def multi_task_train_test_split(Xs,Ys,train_size=0.5):
    Xs_train = []
    Ys_train = []
    Xs_test = []
    Ys_test = []
    for t in range(len(Xs)):
        X_train, X_test, y_train, y_test = train_test_split(Xs[t], np.squeeze(Ys[t]), train_size=train_size)
        Xs_train.append(X_train)
        Xs_test.append(X_test)
        Ys_train.append(y_train)
        Ys_test.append(y_test)
    return Xs_train, Xs_test, Ys_train, Ys_test

In [47]:
T = len(data)
Xs_lm = []
Ys_lm = []
for t in range(len(data)): #nb de taches
    X_t = data[t]
    Xs_lm.append(np.hstack((X_t,np.ones((X_t.shape[0],1)))))
    label2 = labels[t].reshape(500,1) # 500 observations
    Ys_lm.append(label2 == 1.0)
    
d = Xs_lm[0].shape[1]
k = 8

Xs_lm_train, Xs_lm_test, Ys_lm_train, Ys_lm_test = multi_task_train_test_split(Xs_lm,Ys_lm,train_size=0.5)

model = ELLA(d,k,LogisticRegression,{'C':10**0},mu=1,lam=10**-5)
for t in range(T):
    model.fit(Xs_lm_train[t], Ys_lm_train[t], t)

print(model.S )   

print("Average AUC:", np.mean([roc_auc_score(Ys_lm_test[t],model.predict_logprobs(Xs_lm_test[t], t))for t in range(1)]))

[[ 0.06336182  0.          0.          0.         -0.          0.00381675
  -0.00607505 -0.02415788]
 [ 0.11041652  0.          0.          0.13464361 -0.03513807  0.04338136
   0.01450154 -0.00295815]
 [ 0.28485572  0.59018209  0.42143619  0.27200463  0.38857781  0.28054583
   0.29119391  0.37660644]
 [-0.00511405 -0.         -0.         -0.          0.         -0.
  -0.         -0.        ]
 [-0.00544673 -0.         -0.         -0.          0.         -0.
  -0.         -0.30494733]
 [ 0.18848319  0.          0.09849705  0.16336277 -0.05161076  0.02331687
   0.02253345  0.00500433]
 [ 0.05993089  0.          0.          0.         -0.00216503  0.
   0.02257212 -0.01917811]
 [ 0.06917947  0.          0.          0.         -0.05291463 -0.02656126
  -0.03173663 -0.0265936 ]]
Average AUC: 0.7103085509012765


C:\Users\Elvire\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [50]:
general_acc = []
for i in range(20) : 
    print(i)
    classif_accuracy = 0
    #print("Average explained variance score", np.mean([model.score(Xs_lm_test[t], Ys_lm_test[t], t) for t in range(T)]))
    # Try out a classification problem
    Ys_binarized_train = [Ys_lm_train[i] > 0 for i in range(T)]
    Ys_binarized_test = [Ys_lm_test[i] > 0 for i in range(T)]

    model = ELLA(d,k,LogisticRegression,mu=1,lam=10**-5)
    for t in range(T):
        model.fit(Xs_lm_train[t], Ys_binarized_train[t], t)

    classif_accuracy = np.mean([model.score(Xs_lm_test[t], Ys_binarized_test[t], t) for t in range(T)])
    general_acc.append(classif_accuracy)
    i+1

0
1
2
3
4
5


C:\Users\Elvire\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [51]:
print(np.mean(general_acc))
print(general_acc)

0.739325
[0.7375, 0.7384999999999999, 0.7384999999999999, 0.737, 0.742, 0.7404999999999999, 0.7384999999999999, 0.7395, 0.7395, 0.7384999999999999, 0.74, 0.741, 0.74, 0.735, 0.741, 0.7375, 0.7384999999999999, 0.7435, 0.7384999999999999, 0.7415]
